In [ ]:
!pip install -U transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
decision_trace = {
    "decision": "EARLY_BEARING_DEGRADATION",
    "confidence": 0.82,
    "rules_triggered": [
        "VIBRATION_TREND_RULE",
        "THERMAL_CONFIRMATION_RULE"
    ],
    "intermediate_states": {
        "risk_after_vibration": 0.61,
        "risk_after_temperature": 0.82
    }
}

In [ ]:
SEVERITY_MAP = [
    (0.0, "low"),
    (0.4, "moderate"),
    (0.7, "high"),
    (0.9, "critical")
]

DECISION_INTERPRETATIONS = {
    "EARLY_BEARING_DEGRADATION": "internal mechanical wear in rotating components",
    "OVERHEATING": "abnormal thermal behavior",
    "MISALIGNMENT": "shaft or coupling misalignment",
    "LUBRICATION_ISSUE": "insufficient or degraded lubrication"
}

In [ ]:
def score_to_level(score):
    for threshold, label in reversed(SEVERITY_MAP):
        if score >= threshold:
            return label
    return "low"


def score_to_percentage(score):
    return int(score * 100)

In [ ]:
def humanize_decision_trace(trace):
    observations = []

    for key, score in trace.get("intermediate_states", {}).items():
        sensor_name = key.replace("risk_after_", "").replace("_", " ")
        level = score_to_level(score)
        percentage = score_to_percentage(score)

        observations.append(
            f"{sensor_name.capitalize()} indicators increased, reaching a {level} level (around {percentage}%)."
        )

    decision_text = DECISION_INTERPRETATIONS.get(
        trace["decision"],
        "an abnormal machine condition"
    )

    return {
        "observations": observations,
        "assessment": f"The combined sensor behavior indicates {decision_text}.",
        "decision": decision_text.capitalize() + " detected.",
        "confidence": f"{score_to_level(trace['confidence']).capitalize()} confidence ({score_to_percentage(trace['confidence'])}%)."
    }

In [ ]:
human_trace = humanize_decision_trace(decision_trace)

prompt = f"""
You are a mechanical maintenance engineer explaining a machine condition to another engineer.

Rules:
- Use only the information below.
- Explain what the numbers mean physically.
- Do not mention system rules, scores, or internal logic.
- Use cause-and-effect reasoning.
- Write a short, clear paragraph.

Machine observations:
- {human_trace['observations'][0]}
- {human_trace['observations'][1]}

Assessment:
{human_trace['assessment']}

Decision:
{human_trace['decision']}

Confidence:
{human_trace['confidence']}

Write the explanation.
"""

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

outputs = model.generate(
    **inputs,
    max_length=170,
    temperature=0.1,
    num_beams=6,
    repetition_penalty=1.3,
    early_stopping=True
)

explanation = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(explanation)

Vibration indicators increased, reaching a moderate level (around 61%). Temperature indicators increased, reaching a high level (around 82%). The combined sensor behavior indicates internal mechanical wear in rotating components.
